In [222]:
from __future__ import print_function
import os
import numpy as np
from numpy import newaxis
from sklearn import preprocessing
np.random.seed(1337)

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout
from keras.models import Model,Sequential
from keras.layers import Convolution1D, MaxPooling1D
import sys

import pandas as pd
import random

# Reading in data

In [205]:
print("Data read started...")
data = pd.read_csv("nback/result1.csv")
data = data.as_matrix()
print ("Data read finished.")

print(data.shape)

Data read started...
Data read finished.
(622, 19)


# Eliminate EEG data

In [206]:
for i in range (1,5):
    data = np.delete(data, 1, 1) 
    
#data = np.delete(data, 1, 1) 
data = data[:,0:7]
data.shape

(622, 7)

In [207]:
data[0]

array(['3back', 1537.7, 796.38, 36.961999999999996, 4.696727272727268,
       8.947000000000001, 16.387999999999998], dtype=object)

In [208]:
zero = 0
one = 0
two = 0
three = 0

for i in range(len(data)):
    if (data[i, 0] == '0back'):
        zero = zero + 1
    if (data[i, 0] == '1back'):
        one = one + 1
    if (data[i, 0] == '2back'):
        two = two + 1
    if (data[i, 0] == '3back'):
        three = three + 1

print('zero:', zero, ' one: ', one, ' two: ', two, ' three: ', three)

zero: 206  one:  130  two:  139  three:  147


# Dictionary for the levels

In [209]:
level=["0back","1back","2back","3back"]
level2int = dict((p, i) for i, p in enumerate(level))
int2level = dict((i, p) for i, p in enumerate(level))

# Z normalize

In [210]:
for i in range (1,7):
    data[:, i] = preprocessing.scale(data[:, i])

/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConve

In [211]:
back0 = np.zeros((zero,6))
back1 = np.zeros((one,6))
back2 = np.zeros((two,6))
back3 = np.zeros((three,6))
j = 0
k = 0
l = 0
m = 0

for i in range(len(data)):
    if (data[i, 0] == '0back'):
        back0[j] = data[i, 1:]
        j = j+1
    if (data[i, 0] == '1back'):
        back1[k] = data[i, 1:]
        k = k+1
    if (data[i, 0] == '2back'):
        back2[l] = data[i, 1:]
        l = l+1
    if (data[i, 0] == '3back'):
        back3[m] = data[i, 1:]
        m = m+1

In [212]:
y_data = data[:, 0]
for i in range(len(data)):
    y_data[i] = level2int[y_data[i]]

# One-hot encoding

In [213]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
one_hot = ohe.fit_transform(y_data.reshape(-1,1)).toarray()
print(one_hot)

[[ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 ..., 
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]]


# Group data for convolutional network

In [214]:
track = 20
null = np.array([0,0,0,0,0,0])

for i in range(track-1):
        back0 = np.vstack([back0, null])
        back1 = np.vstack([back1, null])
        back2 = np.vstack([back2, null])
        back3 = np.vstack([back3, null])

In [216]:
x_data = np.zeros((data.shape[0],6,track))         # final input data for the network
y_one_hot = np.zeros((data.shape[0],1,4))          # to store one-hot data groupped

for i in range(len(one_hot)):
    y_one_hot[i]=one_hot[i]

for i in range(len(back0)-track+1):
    for j in range(track-1):
        x_data[i, :, j] = back0[i+j]
        
index = 0
for i in range(len(back0)-track+1, len(back0)-track+1 + len(back1)-track+1):
    for j in range(track-1):
        x_data[i, :, j] = back1[index+j]
    index = index+1
    
index = 0
for i in range(len(back0)-track+1 + len(back1)-track+1, len(back0)-track+1 + len(back1)-track+1 + len(back2)-track+1):
    for j in range(track-1):
        x_data[i, :, j] = back2[index+j]
    index = index+1
    
index = 0
for i in range(len(back0)-track+1 + len(back1)-track+1 + len(back2)-track+1, 
                   len(back0)-track+1 + len(back1)-track+1 + len(back2)-track+1 + len(back3)-track+1):
    for j in range(track-1):
        x_data[i, :, j] = back3[index+j]
    index = index+1

# Shuffle data

In [217]:
indices = np.arange(x_data.shape[0])
np.random.shuffle(indices)

x_data = x_data[indices]
y_one_hot = y_one_hot[indices]

# Divide into train, validation and test data

In [218]:
len_data = len(x_data)

nb_test = int(len_data*0.15)
nb_validation = int(len_data*0.15)
nb_train = int(len_data*0.7)

end_valid = nb_train+nb_validation

x_train = x_data[0:nb_train]
y_train = y_one_hot[0:nb_train]

x_valid = x_data[nb_train:end_valid]
y_valid = y_one_hot[nb_train:end_valid]

x_test = x_data[end_valid:]
y_test = y_one_hot[end_valid:]

In [219]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(435, 6, 20) (435, 1, 4) (93, 6, 20) (93, 1, 4) (94, 6, 20) (94, 1, 4)


# Build the net

In [223]:
model = Sequential()

model.add(Convolution1D(input_shape=(x_train[0].shape[-2],x_train[0].shape[-1]),
                        nb_filter=30,
                        filter_length=2,
                        border_mode='same',
                        subsample_length=2,
                        init='glorot_normal',
                        activation='relu')) 
# First max-pooling layer
model.add(MaxPooling1D(pool_length=2,stride=1))
model.add(Dropout(0.5))

# Second convolutional layer
model.add(Convolution1D(nb_filter=30,
                        filter_length=2,
                        border_mode='same',
                        subsample_length=2,
                        init='glorot_normal',
                        activation='relu'))

# Second max-pooling layer
model.add(MaxPooling1D(pool_length=2, stride=1)) 
model.add(Dropout(0.5))

model.add(Flatten())

# Final fully-connected layer
model.add(Dense(4, activation='softmax'))


ValueError: Filter must not be larger than the input: Filter: (2, 1) Input: (1, 1)

In [27]:
batch_size = 50
epochs = 300
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')

model.compile(loss='categorical_crossentropy',optimizer='nadam',metrics=['acc'])
model.fit(x_train, y_train, nb_epoch=epochs,batch_size=batch_size, 
          callbacks=[earlyStopping], shuffle=True, validation_data = (x_valid, y_valid))

Train on 435 samples, validate on 93 samples
Epoch 1/300
435/435 [==============================] - 0s - loss: 1.2869 - acc: 0.4000 - val_loss: 1.1986 - val_acc: 0.4301
Epoch 2/300
435/435 [==============================] - 0s - loss: 1.0749 - acc: 0.5494 - val_loss: 0.9842 - val_acc: 0.5591
Epoch 3/300
435/435 [==============================] - 0s - loss: 0.9629 - acc: 0.6000 - val_loss: 1.0536 - val_acc: 0.5269
Epoch 4/300
435/435 [==============================] - 0s - loss: 0.9264 - acc: 0.6046 - val_loss: 0.9848 - val_acc: 0.5699
Epoch 5/300
435/435 [==============================] - 0s - loss: 0.8987 - acc: 0.6299 - val_loss: 0.9642 - val_acc: 0.6774
Epoch 6/300
435/435 [==============================] - 0s - loss: 0.8728 - acc: 0.6322 - val_loss: 0.9392 - val_acc: 0.6237
Epoch 7/300
435/435 [==============================] - 0s - loss: 0.8385 - acc: 0.6483 - val_loss: 0.9762 - val_acc: 0.6774
Epoch 8/300
435/435 [==============================] - 0s - loss: 0.8444 - acc: 0.6460 

In [28]:
model.evaluate(x_test, y_test, batch_size=batch_size)

50/94 [==============>...............] - ETA: 0s

[0.90714684699444059, 0.75531914386343446]

In [29]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict_classes(x_test)
print(y_pred)

p=model.predict_proba(x_test)

target_names = ['class 0(0back)', 'class 1(1back)', 'class 2(2back)', 'class3(3back)']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

32/94 [=========>....................] - ETA: 0s[0 0 2 2 3 2 0 3 1 1 0 0 0 3 2 1 1 0 3 0 2 2 0 0 1 0 0 3 1 3 0 1 0 2 0 0 0
 2 1 0 2 0 0 0 3 0 2 0 2 2 0 1 3 0 3 0 3 3 0 2 2 0 2 2 2 3 1 3 0 2 0 3 0 2
 2 0 1 2 3 3 2 2 2 0 0 2 1 2 1 0 2 0 0 3]
32/94 [=========>....................] - ETA: 0s                precision    recall  f1-score   support

class 0(0back)       0.70      0.87      0.78        30
class 1(1back)       0.92      0.48      0.63        25
class 2(2back)       0.78      1.00      0.88        21
 class3(3back)       0.71      0.67      0.69        18

   avg / total       0.78      0.76      0.74        94

[[26  1  0  3]
 [ 9 12  2  2]
 [ 0  0 21  0]
 [ 2  0  4 12]]
